# Deploy a Python function
The code in this notebook has been adapted from <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/1eddc77b3a4340d68f762625d40b64f9?context=cpdaas&audience=wdp">Use Python to recognize hand-written digits</a>.

In this notebook, you will create and deploy a Python function that calls an already deployed model. Before editing the notebook, you retrieved the `space_id` and `model_deployment_id` for the model you created in previous steps. It is possible to use Python code in a Jupyter notebook to retrieve this values, but that is outside the scope of this exercise.

Edit the code in the cell below, assigning the correct values to `api_key`, `space_id`, and `model_deployment_id`. Then run the cells one at a time.

In [1]:
api_key = 'GNQrm9fQyayJqA4fsYLD3VYmLgxMCccdKt4KNFjCBtG8'
location = 'us-south'

space_id= "698d3902-0283-4d2b-a8b8-fcca6e458679"
model_deployment_id = "a2999222-260d-4606-a513-021c97c45175"

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

## Install the IBM Watson Machine Learning library

The next cell uses the `pip` command to install the IBM Watson Machine Learning library. Though the library is available by default in all Watson Studio notebooks, the `-U` flag ensures that the most recent version is installed. The the output of the execution is redirected to a file in the notebook's local file systenm to limit what is displayed.

If you really want to see the output, create a new cell and execute the following command:
```
!cat wml_lib_install.out
```

In [3]:
!pip install -U ibm-watson-machine-learning >wml_lib_install.out 2>&1

## Create a wml client connection

To use the Watson Machine Learning (WML) library, you much create a client connection. The code below will create the connection. If you receive an error from running this code, it is likely because you did not correctly assign a valid API key at the beginning of the notebook.

In [4]:
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(wml_credentials)

## Create a function that calls a model

You previously deployed a model named `attrition challenger - spark deployment`. The cell below defines a function called `attrition_function`. It contains two additional functions:
- The `score` function is the execution point in the REST API call, and will call the deployed model.
- The `reformat_output` function will take the output (prediction) from the model, and simplify it by removing some of the fields.

Since both `score` and `reformat_output` are contained within `attrition_function`, `reformat_output` is accessible to `score` when it executes. Run the cell below to create the function definitions.

In [5]:
params = {
    "wml_credentials": wml_credentials, 
    "space_id": space_id, 
    "model_deployment_id": model_deployment_id
}

def attrition_function( parms=params ):
    def reformat_output(result) :
        predictions = []
        for pred in result['predictions'] :
            fields=["probability", "prediction"]
            prob_index = pred["fields"].index("probability")
            pred_index = pred["fields"].index("prediction")
            values = [[item[prob_index][int(item[pred_index])], item[pred_index]] for item in pred["values"]]
            predictions.append( {"fields": fields, "values": values})

        result2 = {
            "predictions": predictions
        }
        return result2
    
    def score(payload):   
        try:                 
            from ibm_watson_machine_learning import APIClient
            client       = APIClient(parms["wml_credentials"])
            client.set.default_space(parms["space_id"])
            model_result = client.deployments.score(parms["model_deployment_id"], payload)
            return reformat_output(model_result)
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
            #return {"error" : repr(e)}
    return score

The cell below tests the function using some sample data. If you get an error message from executing this cell, it is likely because you did not correctly assign the `model_deployment_id` or the `space_id` at the beginning of the notebook. Executing the cell should print out a simplified model prediction containing only the prediction and probability.

In [6]:
# Test locally before deploying. This is useful for debugging
payload = {
    "input_data": [{
        "fields": ["POSITION_CODE","DEPARTMENT_CODE","DAYS_WITH_COMPANY","COMMUTE_TIME",
                   "AGE_BEGIN_PERIOD","GENDER_CODE","PERIOD_TOTAL_DAYS","STARTING_SALARY",
                   "ENDING_SALARY","NB_INCREASES","BONUS","NB_BONUS","VACATION_DAYS_TAKEN",
                   "SICK_DAYS_TAKEN","PROMOTIONS","NB_MANAGERS","DAYS_IN_POSITION",
                   "DAYS_SINCE_LAST_RAISE","RANKING_CODE","OVERTIME","DBLOVERTIME","TRAVEL"],
        "values": [[4500.0,360.0,910.0,22.0,56.0,1.0,330.0,87307.69,94615.38,1.0,
                    5676.9228,1.0,14.0,9.0,0.0,1.0,910.0,180.0,4.0,0.0,0.0,0.0]]
                    }]
}
func_result = attrition_function()(payload)
print(func_result)

{'predictions': [{'fields': ['probability', 'prediction'], 'values': [[0.9145364373814321, 0.0]]}]}


## Deploy the function
Next, you will use the capabilities from the `ibm-watson-machine-learning` to deploy the function to a deployment space. Once the function has been deployed, it will have much of the same functionality as a deployed model; it can be called using a REST endpoint, and can be monitored by Watson Trust (OpenScale). See the <a href="https://cloud.ibm.com/apidocs/machine-learning-cp?code=python">Watson Machine Learning</a> 
documentation for more details.

Run the following two cells to deploy the function.

In [7]:
client.set.default_space(space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'attrition function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=attrition_function)

In [8]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "attrition function",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '2302eb59-5eab-4ea4-8780-2e55fcf887ad' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8ce5d483-2413-4737-b457-ae2914982069'
------------------------------------------------------------------------------------------------




## Test the deployed function

The following cells test the newly-deployed function. Note that it uses information from the `function_deployment_details` variable defined in the cell above while deploying the function. If you were to come directly to this section, you would have to make additional calls to retrieve this information.

In [9]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))

Function deployment id: 8ce5d483-2413-4737-b457-ae2914982069
Endpoint URL: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8ce5d483-2413-4737-b457-ae2914982069/predictions


### Get the data to score

The following cell retreives sample data from github to send to the function. The data is read into memory. 
You could write the content to the notebook's local file system with:
```
r = requests.get(url)
file_name = url.split('/')[-1]
with open(file_name, "wb") as file:
        file.write(r.content)
```
You could then read the data from the notebook's local file system.

In [10]:
# Same file we used in an earlier exercise
url = "https://raw.githubusercontent.com/CloudPak-Outcomes/Outcomes-Projects/main/TrustedAI-L3-Tech-Lab/records_to_score.json"
r = requests.get(url)
payload = json.loads(r.content.decode("utf-8"))

### Execute the function and see the result

In [11]:
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'predictions': [{'fields': ['probability', 'prediction'], 'values': [[0.9145364373814321, 0.0], [0.9566298940491723, 1.0], [0.9655666518862813, 1.0], [0.8896671187930532, 1.0], [0.952045394245465, 1.0]]}]}
